# gemma 2 2b-it 모델 파인튜닝 해보기
https://devocean.sk.com/blog/techBoardDetail.do?ID=165703&boardType=techBlog

    - data_set : nlpai-lab/KULLM - huggingface
    - Model : google/gemma-2-2b-it

In [ ]:
# 다운 받을 모듈 정리
! pip install -q -U transformers datasets bitsandbytes peft trl accelerate

In [1]:
# 모듈 정리 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, GemmaTokenizerFast
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 로딩 및 데이터 확인

In [2]:
# 데이터셋 로드 : KULLM이라는 LLM의 학습 데이터셋 불러오기
from datasets import load_dataset

dataset = load_dataset('nlpai-lab/kullm-v2', split="train")
print("데이터 셋 종류 :", dataset.features.keys())
print("id 종류 :", set(dataset['id']))
for key in list(set(dataset['id'])):
    print(f"{str(key)} 퍼센트 :", round(len([i for i in dataset if i['id'] ==str(key)])/len(dataset) * 100, 2), '%')

데이터 셋 종류 : dict_keys(['id', 'instruction', 'input', 'output'])
id 종류 : {'alpaca_{idx}', 'vicuna_{idx}', 'dolly_{idx}'}
alpaca_{idx} 퍼센트 : 34.07 %
vicuna_{idx} 퍼센트 : 56.1 %
dolly_{idx} 퍼센트 : 9.83 %


In [3]:
# 데이터 예시 
print("dataset['instruction'][0] >>> \n\t", dataset['instruction'][0])
print("dataset['output'][0] >>> \n\t", dataset['output'][0])

dataset['instruction'][0] >>> 
	 3원색이란 무엇인가요?
dataset['output'][0] >>> 
	 세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.


# 데이터 형식 초기화

In [4]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(r"""<bos><start_of_turn>user
사용자의 질문 입니다. 적절한 답변을 해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['instruction'][i], example['output'][i]))
    return prompt_list

generate_prompt(dataset[:2])

['<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n3원색이란 무엇인가요?<end_of_turn>\n<start_of_turn>model\n세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.<end_of_turn><eos>',
 '<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n어려운 결정을 내려야 했던 순간을 설명해 주세요.<end_of_turn>\n<start_of_turn>model\nAI 어시스턴트인 저는 저만의 개인적인 경험이나 물리적 존재를 가지고 있지 않습니다. 하지만 알고리즘, 논리, 데이터를 기반으로 의사 결정을 내리도록 프로그래밍되어 있습니다. 상충되는 정보나 여러 가지 가능한 결과가 있을 수 있기 때문에 의사결정이 어려운 상황에 직면할 때가 있습니다. 그런 경우 저는 각 옵션의 위험과 이점을 분석하고 비교하여 원하는 결과를 얻을 가능성이 가장 높은 결론에 도달할 수 있는 능력을 활용합니다.<end_of_turn><eos>']

# gemma 양자화

In [5]:
# BitsAndBytes 설정: 모델을 4비트로 양자화하여 메모리 사용량을 줄이고 성능 최적화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_quant_type="nf4",  
    bnb_4bit_compute_dtype=torch.float16  
)

# 베이스 모델 
BASE_MODEL = "google/gemma-2-2b-it"

# 토크나이저 설정 
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# 모델 설정 
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, quantization_config=bnb_config,
    device_map="auto",  # 자동으로 GPU에 할당
    attn_implementation='eager' 
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


In [6]:
# LoRA 설정: 대규모 언어 모델의 특정 레이어에서만 파라미터를 미세 조정하여
# 메모리 사용량을 줄이고 학습 효율성을 높임
lora_config = LoraConfig(
    r=6,  
    lora_alpha=8,  
    lora_dropout=0.05,  
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],  
    task_type="CAUSAL_LM",  
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps = 1000,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to yo

In [7]:
### 모델 학습하기 ### 
trainer.train()

Step,Training Loss
100,2.226400
200,1.658800
300,1.513400
400,1.467900
500,1.446200
600,1.442000
700,1.406700
800,1.427800
900,1.437500
1000,1.432800


TrainOutput(global_step=3000, training_loss=1.4303758595784506, metrics={'train_runtime': 7870.7505, 'train_samples_per_second': 1.525, 'train_steps_per_second': 0.381, 'total_flos': 5.066762968044288e+16, 'train_loss': 1.4303758595784506, 'epoch': 0.07862150298106532})

In [8]:
### 모델 저장하기 ###
LORA_SAVE_NAME = "gemma-2-2b-FT-lr"
BASE_MODEL = "google/gemma-2-2b-it"

trainer.model.save_pretrained(LORA_SAVE_NAME)
trainer.tokenizer.save_pretrained(BASE_MODEL)

('gemma-2-2b-FT(feat.kullm)/tokenizer_config.json',
 'gemma-2-2b-FT(feat.kullm)/special_tokens_map.json',
 'gemma-2-2b-FT(feat.kullm)/tokenizer.model',
 'gemma-2-2b-FT(feat.kullm)/added_tokens.json',
 'gemma-2-2b-FT(feat.kullm)/tokenizer.json')

In [5]:
! ls -alh gemma-2-2b-FT\(feat.kullm\)

total 51M
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 24 11:04 .
drwxrwxr-x 7 ubuntu ubuntu 4.0K Sep 24 11:04 ..
-rw-rw-r-- 1 ubuntu ubuntu 5.0K Sep 24 11:04 README.md
-rw-rw-r-- 1 ubuntu ubuntu  721 Sep 24 11:04 adapter_config.json
-rw-rw-r-- 1 ubuntu ubuntu  30M Sep 24 11:04 adapter_model.safetensors
-rw-rw-r-- 1 ubuntu ubuntu  636 Sep 24 11:04 special_tokens_map.json
-rw-rw-r-- 1 ubuntu ubuntu  17M Sep 24 11:04 tokenizer.json
-rw-rw-r-- 1 ubuntu ubuntu 4.1M Sep 24 11:04 tokenizer.model
-rw-rw-r-- 1 ubuntu ubuntu  46K Sep 24 11:04 tokenizer_config.json


In [3]:
! ls -alh ./gemma-2b-it-sum-ko

total 4.9G
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 23 13:06 .
drwxrwxr-x 7 ubuntu ubuntu 4.0K Sep 24 11:04 ..
-rw-rw-r-- 1 ubuntu ubuntu  880 Sep 23 13:06 config.json
-rw-rw-r-- 1 ubuntu ubuntu  187 Sep 23 13:06 generation_config.json
-rw-rw-r-- 1 ubuntu ubuntu 4.7G Sep 23 13:06 model-00001-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu 230M Sep 23 13:06 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu  24K Sep 23 13:06 model.safetensors.index.json


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

### 모델 불러오기 ### 

FINETUNE_MODEL = "gemma-2-2b-FT(feat.kullm)"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(FINETUNE_MODEL)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트로 로드하여 메모리 효율성을 극대화함
    bnb_4bit_quant_type="nf4",  # NF4(Normalized Float 4) 방식의 4비트 양자화 사용
    bnb_4bit_compute_dtype=torch.float16  # 계산에 사용할 데이터 타입을 float16으로 설정 (16비트 부동소수점)
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.62s/it]


NameError: name 'BASE_MODEL' is not defined

In [ ]:

### 추론 ###
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role": "from", "content":"value", "user": "human", "assistant":"gpt"},
)

# 모델을 2배 빠른 추론 모드로 전환
FastLanguageModel.for_inference(model)

# 입력 메시지 설정
text = '오늘의 날씨는 맑았어 좋은 점이 뭘까?'
messages = [
    {
        "role": "user",
        "content": "사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n{}".format(text)
    }
]

# 입력 데이터를 텐서로 변환후 gpu 전송
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt = True,
    return_tensors="pt" # 파이토치 텐서로 변환
).to("cuda")